<center><a href="https://www.nvidia.cn/training/"><img src="https://dli-lms.s3.amazonaws.com/assets/general/DLI_Header_White.png" width="400" height="186" /></a></center>

<br>

# <font color="#76b900">**Notebook 4:** 运行状态链（Running State Chain）</font>

<br>

在上一个 notebook 中，我们介绍了运行时相关的 LangChain Expression Language（LCEL）。到目前为止，您应该已经对内部和外部推理都比较熟悉了，也了解了如何开发推理工作流！在此 notebook 中，我们将进入到更高级的范式，使我们能编排更复杂的对话管理策略、执行长文档推理。

<br>

### **学习目标：**

* 学习利用运行时（Runnable）编排有趣的 LLM 系统。
* 理解运行状态链在对话管理和迭代决策中的作用。

<br>

### **思考问题：**
* 只有一个模块且不持续向环境索要输入的运行状态链变体会有什么用处么？
* 您可能会注意到，JSON 预测的效果是很不错的。当然，根据问题和 JSON 格式的复杂性，它的效果可能并不总是那么好。据此您预计会遇到哪些问题？
* 您能想到哪些可行的方法，把状态链的提示词完全替换掉？

<br>

### **Notebook 版权声明：**

* 本 notebook 是 [**NVIDIA 深度学习培训中心**](https://www.nvidia.cn/training/)的课程[**《构建大语言模型 RAG 智能体》**](https://www.nvidia.cn/training/instructor-led-workshops/building-rag-agents-with-llms/)中的一部分，未经 NVIDIA 授权不得分发。

<br> 


### **环境设置：**

In [ ]:
## Necessary for Colab, not necessary for course environment
# %pip install -q langchain langchain-nvidia-ai-endpoints gradio

# import os
# os.environ["NVIDIA_API_KEY"] = "nvapi-..."

from functools import partial
from rich.console import Console
from rich.style import Style
from rich.theme import Theme

console = Console()
base_style = Style(color="#76B900", bold=True)
pprint = partial(console.print, style=base_style)

In [ ]:
from langchain_nvidia_ai_endpoints import ChatNVIDIA
ChatNVIDIA.get_available_models()

In [ ]:
## Useful utility method for printing intermediate states
from langchain_core.runnables import RunnableLambda
from functools import partial

def RPrint(preface="State: "):
    def print_and_return(x, preface=""):
        print(f"{preface}{x}")
        return x
    return RunnableLambda(partial(print_and_return, preface=preface))

def PPrint(preface="State: "):
    def print_and_return(x, preface=""):
        pprint(preface, x)
        return x
    return RunnableLambda(partial(print_and_return, preface=preface))

----

<br>

## **第 1 部分：** 让变量传递下去

在之前的示例中，我们能通过**创建**、**转变**（mutating）和**消耗**（consuming）状态，在独立的链中实现有趣的逻辑。这些状态以字典的形式进行传递，其中的键值对有实际含义，它们将用于为后续步骤提供操作所需的信息！

**回顾上一个 notebook 中的零样本分类示例：**

In [ ]:
%%time
## ^^ This notebook is timed, which will print out how long it all took

from langchain_core.runnables import RunnableLambda
from langchain_nvidia_ai_endpoints import ChatNVIDIA
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from typing import List, Union
from operator import itemgetter

## Zero-shot classification prompt and chain w/ explicit few-shot prompting
sys_msg = (
    "Choose the most likely topic classification given the sentence as context."
    " Only one word, no explanation.\n[Options : {options}]"
)

zsc_prompt = ChatPromptTemplate.from_template(
    f"{sys_msg}\n\n"
    "[[The sea is awesome]][/INST]boat</s><s>[INST]"
    "[[{input}]]"
)

## Define your simple instruct_model
instruct_chat = ChatNVIDIA(model="mistralai/mistral-7b-instruct-v0.2")
instruct_llm = instruct_chat | StrOutputParser()
one_word_llm = instruct_chat.bind(stop=[" ", "\n"]) | StrOutputParser()


## Function that just prints out the first word of the output. With early stopping bind
zsc_chain = zsc_prompt | one_word_llm

## Function that just prints out the first word of the output. With early stopping bind
def zsc_call(input, options=["car", "boat", "airplane", "bike"]):
    return zsc_chain.invoke({"input" : input, "options" : options}).split()[0]

print("-" * 80)
print(zsc_call("Should I take the next exit, or keep going to the next one?"))

print("-" * 80)
print(zsc_call("I get seasick, so I think I'll pass on the trip"))

print("-" * 80)
print(zsc_call("I'm scared of heights, so flying probably isn't for me"))

这个链做出了几个使其易于使用的设计决策，最关键的如下：

**我们希望它像函数一样，因此我们只需要它能生成并返回输出。**

这使得该链能很自然地成为更大的链系统中的一个模块。比如，以下链将接受字符串，提取最有可能的主题，然后根据主题生成新句子：

In [ ]:
%%time
## ^^ This notebook is timed, which will print out how long it all took
gen_prompt = ChatPromptTemplate.from_template(
    "Make a new sentence about the the following topic: {topic}. Be creative!"
)

gen_chain = gen_prompt | instruct_llm

input_msg = "I get seasick, so I think I'll pass on the trip"
options = ["car", "boat", "airplane", "bike"]

chain = (
    ## -> {"input", "options"}
    {'topic' : zsc_chain}
    | RPrint()
    ## -> {**, "topic"}
    | gen_chain
    ## -> string
)

chain.invoke({"input" : input_msg, "options" : options})

<br>

但如果您想让信息能够继续向下游传递，就会出问题，因为我们在生成响应时丢失了主题和输入变量。如果我们想同时根据输出和输入执行某些操作，就需要确保这两个变量都能传递下去。

幸运的是，我们可以用运行时映射（从字典构建或者直接用 RunnableMap），将链的输出也分配给一个键（key），随着其它键一起传递下去。或者也可以用 `RunnableAssign`，它默认情况下会将需要消耗状态的链的输出与输入字典合并起来。

借助这种技巧，我们就能通过链系统传递任何内容了：

In [ ]:
%%time
## ^^ This notebook is timed, which will print out how long it all took

from langchain.schema.runnable import RunnableBranch, RunnablePassthrough
from langchain.schema.runnable.passthrough import RunnableAssign
from functools import partial

big_chain = (
    PPrint()
    ## Manual mapping. Can be useful sometimes and inside branch chains
    | {'input' : lambda d: d.get('input'), 'topic' : zsc_chain}
    | PPrint()
    ## RunnableAssign passing. Better for running state chains by default
    | RunnableAssign({'generation' : gen_chain})
    | PPrint()
    ## Using the input and generation together
    | RunnableAssign({'combination' : (
        ChatPromptTemplate.from_template(
            "Consider the following passages:"
            "\nP1: {input}"
            "\nP2: {generation}"
            "\n\nCombine the ideas from both sentences into one simple one."
        )
        | instruct_llm
    )})
)

output = big_chain.invoke({
    "input" : "I get seasick, so I think I'll pass on the trip",
    "options" : ["car", "boat", "airplane", "bike", "unknown"]
})
pprint("Final Output: ", output)

----

<br>

## **第 2 部分：** 运行状态链

上面只是一个示例，它展示出了将许多 LLM 调用连接在一起进行内部推理时的缺陷。但让信息在链中传递的能力，对那些需要积累状态信息、或多次执行的复杂链至关重要。

具体来说，一个非常简单但有效的链是**运行状态链**，它有以下属性：
* “运行状态”（running state）是一个包含了系统关注的所有变量的字典。
* “分支”（branch）是一条可以拉取运行状态并生成响应的链。
* “分支”只能在 **RunnableAssign** 内运行，分支的输入应来自**运行状态**。

> <img src="https://dli-lms.s3.amazonaws.com/assets/s-fx-15-v1/imgs/running_state_chain.png" width=1000px/>

您可以抽象地将运行状态链看作 Python 类的变体，其中包含状态变量（或者说属性）和函数（或者说方法）。
* 这种链像一个囊括了所有功能的抽象类。
* 运行状态类似于属性（应使其始终可访问）。
* 分支类似于类方法（可以选择要用的属性）。
* `.invoke` 或类似的方法就像是按顺序在分支中运行 `__call__` 方法。

**通过在链中强制实行此范式：**
* 您可以让状态变量在链中传播，允许内部逻辑访问任何需要的内容，并累积状态值以供之后使用。
* 您还可以将链的输出再次作为您的输入，从而创建出一个“while-loop”式的根据运行状态不断构建的链。

此 notebook 的剩余部分将包括两个练习，它们能很好的体现运行状态链抽象：**知识库**和**数据库查询聊天机器人**。

----

<br>

## **第 3 部分：** 使用运行状态链实现知识库

在了解了运行状态链的基本结构和原理后，我们就可以探索怎么扩展此方法以管理更复杂的任务了，尤其是在创建通过交互而演进的动态系统时。本节将重点介绍如何通过 **JSON 格式的槽位填充**（JSON-Enabled Slot Filling）实现**知识库**：

* **知识库：**存储 LLM 需要关注的信息的存储库。
* **JSON 格式的槽位填充：**要求指令微调过的模型输出带有槽位的 JSON 格式（可以包括字典），再用 LLM 为这些槽位填充有用的相关信息。

<br>

#### **定义知识库**

为了构建一个高响应的智能系统，我们需要一种不仅能处理输入，还能通过对话流保留和更新基本信息的方法。这正是我们需要结合 LangChain 和 Pydantic 的地方。作为热门的 Python 验证库，[**Pydantic**](https://docs.pydantic.dev/latest/) 在构建和验证数据模型方面发挥了重要作用。Pydantic 的功能之一是提供结构化的“模型（model）”类，这些类可以通过简洁的语法和自定义来验证数据、类等对象。此框架在整个 LangChain 中都有广泛使用，是数据转换的必要组件。

“模型”擅长的就是定义一个包含预期参数的类，并能通过一些特别的方法验证它们！我们不会在课程中介绍过多关于验证脚本的情况，感兴趣的学员可以查看 [**Pydantic 验证器指南**](https://docs.pydantic.dev/1.10/usage/validators/)（很快就会变得深入起来）。我们只需要知道怎么用就可以了，可以像下面所示，构建一个 `BaseModel` 类，再定义一些 `Field` 变量来创建结构化知识库：

In [ ]:
from langchain.pydantic_v1 import BaseModel, Field
from typing import Dict, Union, Optional

class KnowledgeBase(BaseModel):
    ## Fields of the BaseModel, which will be validated/assigned when the knowledge base is constructed
    topic: str = Field('general', description="Current conversation topic")
    user_preferences: Dict[str, Union[str, int]] = Field({}, description="User preferences and choices")
    session_notes: str = Field("", description="Notes on the ongoing session")
    unresolved_queries: list = Field([], description="Unresolved user queries")
    action_items: list = Field([], description="Actionable items identified during the conversation")

print(repr(KnowledgeBase(topic = "Travel")))

<br>  

这种方式的优势在于，我们可以集成 LangChain 提供的其它 LLM 功能。其中一个就是 `PydanticOutputParser`，它可以根据 Pydantic 对象自动生成指令。

In [ ]:
from langchain.output_parsers import PydanticOutputParser

instruct_string = PydanticOutputParser(pydantic_object=KnowledgeBase).get_format_instructions()
pprint(instruct_string)

<br>

此功能会生成知识库能接收的有效输入指令，帮助为 LLM 提供具体的单样本学习样例。

<br>

#### **可运行的提取模块**

现在我们已经知道可以用 Pydantic 对象来生成良好的 LLM 指令，那就可以创建一个封装了 Pydantic 类的运行时，构建从提示词、生成结果，到更新知识库的流水线了：

In [ ]:
################################################################################
## Definition of RExtract
def RExtract(pydantic_class, llm, prompt):
    '''
    Runnable Extraction module
    Returns a knowledge dictionary populated by slot-filling extraction
    '''
    parser = PydanticOutputParser(pydantic_object=pydantic_class)
    instruct_merge = RunnableAssign({'format_instructions' : lambda x: parser.get_format_instructions()})
    def preparse(string):
        if '{' not in string: string = '{' + string
        if '}' not in string: string = string + '}'
        string = (string
            .replace("\\_", "_")
            .replace("\n", " ")
            .replace("\]", "]")
            .replace("\[", "[")
        )
        # print(string)  ## Good for diagnostics
        return string
    return instruct_merge | prompt | llm | preparse | parser

################################################################################
## Practical Use of RExtract

parser_prompt = ChatPromptTemplate.from_template(
    "Update the knowledge base: {format_instructions}. Only use information from the input."
    "\n\nNEW MESSAGE: {input}"
)

extractor = RExtract(KnowledgeBase, instruct_llm, parser_prompt)

knowledge = extractor.invoke({'input' : "I love flowers so much! The orchids are amazing! Can you buy me some?"})
pprint(knowledge)

<br>

提醒一点，由于 LLM 预测的模糊性，这个过程可能会失败，尤其是未经指令优化的模型！这个时候，有一个擅长遵循指令的 LLM 、加入额外的检查和纠错机制是非常重要的。

<br>  

#### **动态更新知识库**

最后，我们可以创建一个能在整个对话过程中持续更新知识库的系统。这是通过将知识库的当前状态及用户输入持续反馈给系统来实现的。

以下是一个示例，展示了该方案填充细节的能力，也能看出让模型做填充相较于完成一般响应任务的局限性：

In [ ]:
class KnowledgeBase(BaseModel):
    firstname: str = Field('unknown', description="Chatting user's first name, unknown if unknown")
    lastname: str = Field('unknown', description="Chatting user's last name, unknown if unknown")
    location: str = Field('unknown', description="Where the user is located")
    hints: str = Field('unknown', description="Hints to help answer other questions")
    response: str = Field('unknown', description="An ideal response to the user based on their new message")


parser_prompt = ChatPromptTemplate.from_template(
    "You are chatting with a user. The user just responded. Please update the knowledge base based on the response."
    " This information will be acted on to respond to the user in the next interaction."
    " Do not hallucinate any details, and make sure the knowledge base is not redundant."
    " Update the entries frequently to adapt to the conversation flow."
    "\n{format_instructions}"
    "\n\nOLD KNOWLEDGE BASE: {know_base}"
    "\n\nNEW MESSAGE: {input}"
    "\n\nNEW KNOWLEDGE BASE:"
)

## Switch to a more powerful base model
instruct_llm = ChatNVIDIA(model="mistralai/mixtral-8x22b-instruct-v0.1") | StrOutputParser()

extractor = RExtract(KnowledgeBase, instruct_llm, parser_prompt)
info_update = RunnableAssign({'know_base' : extractor})

## Initialize the knowledge base and see what you get
state = {'know_base' : KnowledgeBase()}
state['input'] = "My name is Carmen Sandiego! Guess where I am! Hint: It's somewhere in the United States."
state = info_update.invoke(state)
pprint(state)

In [ ]:
state['input'] = "I'm in a place considered the birthplace of Jazz."
state = info_update.invoke(state)
pprint(state)

In [ ]:
state['input'] = "Yeah, I'm in New Orleans... How did you know?"
pprint(state)

<br>

这个例子就演示了如何有效地利用运行状态链来处理不断变化上下文和需求的对话，展示了它如何成为开发复杂交互式系统的强大工具。

此 notebook 的下个部分将通过探索两个应用程序来扩展这些概念：**文档知识库**和**数据库查询聊天机器人**。

----

<br>

## **第 4 部分: [练习]** Airline 客户服务机器人

在本练习中，我们来扩展所学的工具，实现一个简单有效的对话管理器聊天机器人。我们将创建一个帮客户了解航班情况的航空公司客服机器人！

先创建一个简单的数据库，用字典获取一些客户信息！

In [ ]:
#######################################################################################
## Function that can be queried for information. Implementation details not important
def get_flight_info(d: dict) -> str:
    """
    Example of a retrieval function which takes a dictionary as key. Resembles SQL DB Query
    """
    req_keys = ['first_name', 'last_name', 'confirmation']
    assert all((key in d) for key in req_keys), f"Expected dictionary with keys {req_keys}, got {d}"

    ## Static dataset. get_key and get_val can be used to work with it, and db is your variable
    keys = req_keys + ["departure", "destination", "departure_time", "arrival_time", "flight_day"]
    values = [
        ["Jane", "Doe", 12345, "San Jose", "New Orleans", "12:30 PM", "9:30 PM", "tomorrow"],
        ["John", "Smith", 54321, "New York", "Los Angeles", "8:00 AM", "11:00 AM", "Sunday"],
        ["Alice", "Johnson", 98765, "Chicago", "Miami", "7:00 PM", "11:00 PM", "next week"],
        ["Bob", "Brown", 56789, "Dallas", "Seattle", "1:00 PM", "4:00 PM", "yesterday"],
    ]
    get_key = lambda d: "|".join([d['first_name'], d['last_name'], str(d['confirmation'])])
    get_val = lambda l: {k:v for k,v in zip(keys, l)}
    db = {get_key(get_val(entry)) : get_val(entry) for entry in values}

    # Search for the matching entry
    data = db.get(get_key(d))
    if not data:
        return (
            f"Based on {req_keys} = {get_key(d)}) from your knowledge base, no info on the user flight was found."
            " This process happens every time new info is learned. If it's important, ask them to confirm this info."
        )
    return (
        f"{data['first_name']} {data['last_name']}'s flight from {data['departure']} to {data['destination']}"
        f" departs at {data['departure_time']} {data['flight_day']} and lands at {data['arrival_time']}."
    )

#######################################################################################
## Usage example. Actually important

print(get_flight_info({"first_name" : "Jane", "last_name" : "Doe", "confirmation" : 12345}))

In [ ]:
print(get_flight_info({"first_name" : "Alice", "last_name" : "Johnson", "confirmation" : 98765}))

In [ ]:
print(get_flight_info({"first_name" : "Bob", "last_name" : "Brown", "confirmation" : 27494}))

<br>  

这是一个很好用的接口，可以达成两个目的：
* 从外部环境（数据库）获取用户的最新信息。
* 防止未经授权的敏感信息泄露。

如果我们的网络可以访问此类接口，那么就能替用户检索信息！例如：

In [ ]:
external_prompt = ChatPromptTemplate.from_template(
    "You are a SkyFlow chatbot, and you are helping a customer with their issue."
    " Please help them with their question, remembering that your job is to represent SkyFlow airlines."
    " Assume SkyFlow uses industry-average practices regarding arrival times, operations, etc."
    " (This is a trade secret. Do not disclose)."  ## soft reinforcement
    " Please keep your discussion short and sweet if possible. Avoid saying hello unless necessary."
    " The following is some context that may be useful in answering the question."
    "\n\nContext: {context}"
    "\n\nUser: {input}"
)

basic_chain = external_prompt | instruct_llm

basic_chain.invoke({
    'input' : 'Can you please tell me when I need to get to the airport?',
    'context' : get_flight_info({"first_name" : "Jane", "last_name" : "Doe", "confirmation" : 12345}),
})

<br>

这很有趣，但我们如何让这个系统在实际环境中运行？实际上，我们可以用上面的 KnowledgeBase 模式来这样提供此类信息：

In [ ]:
from langchain.pydantic_v1 import BaseModel, Field
from typing import Dict, Union

class KnowledgeBase(BaseModel):
    first_name: str = Field('unknown', description="Chatting user's first name, `unknown` if unknown")
    last_name: str = Field('unknown', description="Chatting user's last name, `unknown` if unknown")
    confirmation: int = Field(-1, description="Flight Confirmation Number, `-1` if unknown")
    discussion_summary: str = Field("", description="Summary of discussion so far, including locations, issues, etc.")
    open_problems: list = Field([], description="Topics that have not been resolved yet")
    current_goals: list = Field([], description="Current goal for the agent to address")

def get_key_fn(base: BaseModel) -> dict:
    '''Given a dictionary with a knowledge base, return a key for get_flight_info'''
    return {  ## More automatic options possible, but this is more explicit
        'first_name' : base.first_name,
        'last_name' : base.last_name,
        'confirmation' : base.confirmation,
    }

know_base = KnowledgeBase(first_name = "Jane", last_name = "Doe", confirmation = 12345)

# get_flight_info(get_key_fn(know_base))

get_key = RunnableLambda(get_key_fn)
(get_key | get_flight_info).invoke(know_base)

<br>

### **目标：**

您希望用户能够在交互对话中灵活地调用以下函数：

```python
get_flight_info({"first_name" : "Jane", "last_name" : "Doe", "confirmation" : 12345}) ->
    "Jane Doe's flight from San Jose to New Orleans departs at 12:30 PM tomorrow and lands at 9:30 PM."
```

`RExtract` 用来使以下知识库语法能被接受：
```python
known_info = KnowledgeBase()
extractor = RExtract(KnowledgeBase, InstructLLM(), parser_prompt)
results = extractor.invoke({'info_base' : known_info, 'input' : 'My message'})
known_info = results['info_base']
```

**设计可以实现以下功能的聊天机器人：**
* 机器人应先进行闲聊，或许可以帮助用户处理一些不需要询问隐私信息的非敏感查询。
* 当用户开始询问数据库中的内容时，请让用户提供必要的信息。
* 检索成功后，智能体将能够围绕数据库中的信息继续展开对话。

**这可以通过各种技术来完成，包括：**
* **提示工程和上下文解析（Context Parsing）**，整体的聊天提示词保持大致相同，但可以通过操纵上下文来改变智能体行为。比如，可以通过注入自然语言指令来调整失败的数据库检索过程：*`"Information could not be retrieved with keys {...}. Please ask the user for clarification or help them with known information."`*
* **“提示词传递”**（Prompt Passing），将当前的提示词作为状态变量传递，并能用链中的信息覆盖。
* **分支链**（Branching chains），例如用 [`RunnableBranch`](https://python.langchain.com/docs/expression_language/how_to/routing) 或更多定制方案来实现特定条件的路由机制。
	+ 这是 [`RunnableBranch`](https://python.langchain.com/docs/expression_language/how_to/routing) 类似 `switch` 的语法： 
	```python
	from langchain.schema.runnable import RunnableBranch
	RunnableBranch(
	    ((lambda x: 1 in x), RPrint("Has 1 (didn't check 2): ")),
	    ((lambda x: 2 in x), RPrint("Has 2 (not 1 though): ")),
	    RPrint("Has neither 1 not 2: ")
	).invoke([2, 1, 3]);  ## -> Has 1 (didn't check 2): [2, 1, 3]
	```

提供一些提示词和 gradio 循环可能会有所帮助，但智能体目前仍然只会产生幻觉！请通过实现内部的链来尝试检索相关信息。在尝试之前，可以观察一下模型的默认行为，看看它是怎么产生幻觉或忘记东西的。

In [ ]:
from langchain.schema.runnable import (
    RunnableBranch,
    RunnableLambda,
    RunnableMap,       ## Wrap an implicit "dictionary" runnable
    RunnablePassthrough,
)
from langchain.schema.runnable.passthrough import RunnableAssign

from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import BaseMessage, SystemMessage, ChatMessage, AIMessage
from typing import Iterable
import gradio as gr

external_prompt = ChatPromptTemplate.from_messages([
    ("system", (
        "You are a chatbot for SkyFlow Airlines, and you are helping a customer with their issue."
        " Please chat with them! Stay concise and clear!"
        " Your running knowledge base is: {know_base}."
        " This is for you only; Do not mention it!"
        " \nUsing that, we retrieved the following: {context}\n"
        " If they provide info and the retrieval fails, ask to confirm their first/last name and confirmation."
        " Do not ask them any other personal info."
        " If it's not important to know about their flight, do not ask."
        " The checking happens automatically; you cannot check manually."
    )),
    ("assistant", "{output}"),
    ("user", "{input}"),
])

##########################################################################
## Knowledge Base Things

class KnowledgeBase(BaseModel):
    first_name: str = Field('unknown', description="Chatting user's first name, `unknown` if unknown")
    last_name: str = Field('unknown', description="Chatting user's last name, `unknown` if unknown")
    confirmation: Optional[int] = Field(None, description="Flight Confirmation Number, `-1` if unknown")
    discussion_summary: str = Field("", description="Summary of discussion so far, including locations, issues, etc.")
    open_problems: str = Field("", description="Topics that have not been resolved yet")
    current_goals: str = Field("", description="Current goal for the agent to address")

parser_prompt = ChatPromptTemplate.from_template(
    "You are a chat assistant representing the airline SkyFlow, and are trying to track info about the conversation."
    " You have just recieved a message from the user. Please fill in the schema based on the chat."
    "\n\n{format_instructions}"
    "\n\nOLD KNOWLEDGE BASE: {know_base}"
    "\n\nASSISTANT RESPONSE: {output}"
    "\n\nUSER MESSAGE: {input}"
    "\n\nNEW KNOWLEDGE BASE: "
)

## Your goal is to invoke the following through natural conversation
# get_flight_info({"first_name" : "Jane", "last_name" : "Doe", "confirmation" : 12345}) ->
#     "Jane Doe's flight from San Jose to New Orleans departs at 12:30 PM tomorrow and lands at 9:30 PM."

chat_llm = ChatNVIDIA(model="meta/llama3-70b-instruct") | StrOutputParser()
instruct_llm = ChatNVIDIA(model="mistralai/mixtral-8x22b-instruct-v0.1") | StrOutputParser()

external_chain = external_prompt | chat_llm

#####################################################################################
## START TODO: Define the extractor and internal chain to satisfy the objective

## TODO: Make a chain that will populate your knowledge base based on provided context
knowbase_getter = lambda x: KnowledgeBase()

## TODO: Make a chain to pull d["know_base"] and outputs a retrieval from db
database_getter = lambda x: "Not implemented"

## These components integrate to make your internal chain
internal_chain = (
    RunnableAssign({'know_base' : knowbase_getter})
    | RunnableAssign({'context' : database_getter})
)

## END TODO
#####################################################################################

state = {'know_base' : KnowledgeBase()}

def chat_gen(message, history=[], return_buffer=True):

    ## Pulling in, updating, and printing the state
    global state
    state['input'] = message
    state['history'] = history
    state['output'] = "" if not history else history[-1][1]

    ## Generating the new state from the internal chain
    state = internal_chain.invoke(state)
    print("State after chain run:")
    pprint({k:v for k,v in state.items() if k != "history"})
    
    ## Streaming the results
    buffer = ""
    for token in external_chain.stream(state):
        buffer += token
        yield buffer if return_buffer else token

def queue_fake_streaming_gradio(chat_stream, history = [], max_questions=8):

    ## Mimic of the gradio initialization routine, where a set of starter messages can be printed off
    for human_msg, agent_msg in history:
        if human_msg: print("\n[ Human ]:", human_msg)
        if agent_msg: print("\n[ Agent ]:", agent_msg)

    ## Mimic of the gradio loop with an initial message from the agent.
    for _ in range(max_questions):
        message = input("\n[ Human ]: ")
        print("\n[ Agent ]: ")
        history_entry = [message, ""]
        for token in chat_stream(message, history, return_buffer=False):
            print(token, end='')
            history_entry[1] += token
        history += [history_entry]
        print("\n")

## history is of format [[User response 0, Bot response 0], ...]
chat_history = [[None, "Hello! I'm your SkyFlow agent! How can I help you?"]]

## Simulating the queueing of a streaming gradio interface, using python input
queue_fake_streaming_gradio(
    chat_stream = chat_gen,
    history = chat_history
)

In [ ]:
# state = {'know_base' : KnowledgeBase()}

# chatbot = gr.Chatbot(value=[[None, "Hello! I'm your SkyFlow agent! How can I help you?"]])
# demo = gr.ChatInterface(chat_gen, chatbot=chatbot).queue().launch(debug=True, share=True)

<br>

----

<br>

**注意:**
* 如果 Gradio 界面在出现异常后挂起，您可能需要单击“STOP”并尝试重启。这是已知的 Jupyter Notebook 环境问题，不会在专门的 Gradio 运行文件中遇到。
* **此处复制了您的聊天指令，以便快速访问：**
```
## Your goal is to invoke the following through natural conversation
get_flight_info({
    "first_name" : "Jane",
    "last_name" : "Doe",
    "confirmation" : 12345,
}) -> "Jane Doe's flight from San Jose to New Orleans departs at 12:30 PM tomorrow and lands at 9:30 PM."
```
* **要确认您的系统是否正常工作，可以尝试以下对话：**
```
> How's it going?
> Can you tell me a bit about skyflow?
> Can you tell me about my flight?
> My name is Jane Doe and my flight confirmation is 12345
> Can you tell me when I should get to my flight?
```

* **您可以在“Solutions”目录中找到练习的答案。**这是第一个提供了答案的练习，后续的 notebook 中的练习答案也可以从该目录找到。

-----

<br>

## **第 5 部分：** 总结

此 notebook 的目标是介绍一些更前沿的 LangChain 知识：知识库和运行状态链！这里的练习有一定的深度，恭喜您完成了任务！

### <font color="#76b900">**非常好！**</font>

### **接下来：**
**[可选]** 回顾 notebook 顶部的“思考问题”。